<a href="https://colab.research.google.com/github/bingblackbean/supply_chain_site_selection/blob/main/Center_of_Gravity_for_Multi_site_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
file = '/content/drive/MyDrive/02_data_analysis/data_set/china_cities.csv'


In [106]:
import pandas as pd
random_seed = 2021
all_cities_df = pd.read_csv(file,encoding='gb18030')
customer_df = all_cities_df.sample(n=1000,random_state=random_seed)
customer_df.columns = ['id','name','lon','lat']

In [107]:
from random import randint, seed,gauss,choice
N = customer_df.shape[0]
seed(2)
customer_df['vol'] = [choice([10*(1+i) for i in range(10)]) for _ in range(N)]

print(customer_df)

          id       name         lon        lat  vol
2089  320585     江苏省太仓市  121.158978  31.571904   10
2129  320982     江苏省大丰市  120.546274  33.190868   20
3249  542225      西藏琼结县   91.613770  29.011722   20
805   510500     四川省泸州市  105.443970  28.895930   60
575   450221      广西柳江县  109.274221  24.209539   30
...      ...        ...         ...        ...  ...
2632  361128     江西省鄱阳县  116.787693  29.243056   30
1306  140221     山西省阳高县  113.823181  40.222312   30
815   510601  四川省德阳市市辖区  104.402398  31.131140   70
1496  150727  内蒙古新巴尔虎右旗  114.415868  43.468238   30
2410  341825     安徽省旌德县  118.482898  30.321833   80

[1000 rows x 5 columns]


In [108]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
go.Scattermapbox(mode='markers', 
                                 lon = customer_df['lon'],
                                 lat = customer_df['lat'],
                                 hovertext = customer_df['name'],
                                 hoverinfo = 'text',
                                 marker=go.scattermapbox.Marker(
            size=customer_df['vol']*0.2
        ),
                                 )
)

fig.update_layout(    hovermode='closest',mapbox={'accesstoken':map_box_token,'center':{'lon':customer_df['lon'].values[0],'lat':customer_df['lat'].values[0]},'zoom':3.6},
                 title={'text':'大客户分布','xref':'paper','x':0.5},autosize=False,
    width=1500,
    height=1000,

    )
fig.show()

In [109]:
from scipy.optimize import minimize
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Bing")

def geo_distance(p1: pd.Series,p2:list):
    dis = haversine_distances([[radians(_) for _ in p1.values], [radians(_) for _ in p2]])[0][1]* 6371000/1000  # multiply by Earth radius to get kilometers
    return dis


def cost_func_multi_dc(dc_pos_list,customer_df):
    dc_pos_list = [dc_pos_list[i:i+2] for i in range(0, len(dc_pos_list), 2)]
    for i,dc_pos in enumerate(dc_pos_list):
        customer_df[f'dis_{i}']=customer_df[['lat','lon']].apply(geo_distance,p2=dc_pos,axis=1)
    customer_df['dis_min'] = customer_df[[f'dis_{i}' for i in range(len(dc_pos_list))]].min(axis=1)
    
    customer_df['cost'] = customer_df['dis_min'] * customer_df['vol']
    cost_sum = customer_df['cost'].sum()
    print(dc_pos_list)
    return cost_sum
Dc_Qt = 8
bnds = [(20, 45), (60, 130)]*Dc_Qt
#inits = customer_df.loc[customer_df.index[0],['lat','lon']].to_list()*Dc_Qt
inits = [20,112]*Dc_Qt
res = minimize(cost_func_multi_dc,inits , method='Powell',args=(customer_df),bounds=bnds)

/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:522: RuntimeWarning:

Method Powell cannot handle constraints nor bounds.



[array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.])]
[array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.])]
[array([ 21., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.])]
[array([ 22.618034, 112.      ]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.])]
[array([ 46.48571656, 112.        ]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.])]
[array([ 85.10443845, 112.        ]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array([ 20., 112.]), array

In [110]:
import numpy as np
dc_scipy_df = pd.DataFrame(np.reshape(res['x'],(Dc_Qt,2)),columns=['lat','lon'])

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="baiduv3")
def get_city(df):
    df['location'] = geolocator.reverse(str(df['lat'])+","+str(df['lon'])).address
    return df
dc_scipy_df = dc_scipy_df.apply(get_city, axis=1)
dc_scipy_df

,lat,lon,location
0,35.487223,106.802651,"下甲, 崆峒区, 平凉市, 甘肃省, 中国"
1,44.828934,125.521683,"黄鱼圈乡, 长春市, 吉林省, 130000, 中国"
2,27.903682,102.494010,"西昌市, 凉山彝族自治州, 四川省, 615000, 中国"
3,31.283579,118.253810,"保定街道, 弋江区, 芜湖市, 安徽省, 中国"
4,42.210074,83.376697,"阿克苏地区, 温宿县, 新疆维吾尔自治区, 中国"
5,27.048016,110.728530,"洞口县, 邵阳市, 湖南省, 中国"
6,37.353925,115.259075,"南宫市, 邢台市, 河北省, 054000, 中国"
7,23.889040,114.148571,"X220, 龙门县, 惠州市, 广东省, 中国"


In [111]:
for i,dc_pos in enumerate(dc_scipy_df[['lat','lon']].values):
    customer_df[f'dis_{i}']=customer_df[['lat','lon']].apply(geo_distance,p2=dc_pos,axis=1)
customer_df['dis_min'] = customer_df[[f'dis_{i}' for i in range(Dc_Qt)]].min(axis=1)
customer_df['dc'] = customer_df[[f'dis_{i}' for i in range(Dc_Qt)]].idxmin(axis=1)

In [112]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
go.Scattermapbox(mode='markers', 
                                 lon = customer_df['lon'],
                                 lat = customer_df['lat'],
                                 hovertext = customer_df['name'],
                                 hoverinfo = 'text',
                                 marker=go.scattermapbox.Marker(
            size=customer_df['vol']*0.1,color = customer_df['dc'].astype('category').cat.codes
        ),
                                 )
)
fig.add_trace(go.Scattermapbox(mode='markers', 
                                 lon = dc_scipy_df['lon'],
                                 lat = dc_scipy_df['lat'],
                               hovertext = dc_scipy_df['location'],
                                 hoverinfo = 'text',
                                 marker=go.scattermapbox.Marker(
            size=20,color='green',symbol='castle'
        ),
                                 ))
fig.update_layout(    hovermode='closest',mapbox={'accesstoken':map_box_token,'center':{'lon':customer_df['lon'].values[0],'lat':customer_df['lat'].values[0]},'zoom':3.6},
                 title={'text':'大客户分布','xref':'paper','x':0.5},autosize=False,
    width=1500,
    height=1000,

    )
fig.show()

In [113]:
customer_df['dc'].value_counts()
distance_threshold = 600
calc_service_level = (sum(customer_df['dis_min'] <=distance_threshold))/customer_df.shape[0]
calc_service_level

0.915